In [79]:
import ast
import binascii
import gzip
import io
import json
import os
import pathlib
import tarfile
import zipfile

# import snifbytes as snif
# import load_utils as lu
# import scanzip as szip

from io import BytesIO
from io import StringIO
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join as pjoin

import numpy as np
import pandas as pd
from pandas import DataFrame as df

from pathlib import Path
datalad_config_path = Path(xpu('~/../../data/cisl/DATA/cimaq_20190901/.datalad/config'))


In [2]:
def get_fulltarcontents(mytar):
    return pd.concat([df.from_dict(tarmember.get_info(),
                                   orient='index').T
                              for tarmember in
                              mytar.getmembers()]).sort_values(
                        'name').reset_index(drop = True).iloc[1:220,:]

In [31]:
def get_series_uids(mytar):
    return pd.concat([df.from_dict(tarmember.get_info(),
                                   orient='index').T
                              for tarmember in
                              mytar.getmembers()]).sort_values(
                        'name').reset_index(drop = True).iloc[-1,:]['name']

In [ ]:
# getting clean credentials

name, value = datalad_config_path.read_text().splitlines()
newname=name.replace(']', '').replace(
            '[', '').replace('"', '').replace(' ', '_')
newvalue = value[value.find('= ')+2:]
datalad_credentials = ast.literal_eval(''.join(['{"', str(newname),
                          '"', ' : "', newvalue, '"}']))
with open(xpu('~/Documents/datalad_credentials.json'), 'w') as jfile:
    json.dump(fp=jfile,
              obj=datalad_credentials)
jfile.close()


In [82]:
extractedpath = '~/mytar_extracted/cimaq_dicoms_20190901/'
seriesuids_test = next((pjoin(extractedpath, itm)
                        for itm in ls(extractedpath)))

series_uids = pd.read_csv(StringIO(Path(
                    seriesuids_test).read_text()),
                          sep = '\t')

series_uids

,PSCID,CandID,Visit,Archive,SeriesUID,EchoTime,QC
0,8192087,619278,V03,DCM_2015-05-08_ImagingUpload-14-14-grO2zP.tar,1.3.12.2.1107.5.2.32.35049.2015050810170216329...,0.00298,Pass
1,8192087,619278,V03,DCM_2015-05-08_ImagingUpload-14-14-grO2zP.tar,1.3.12.2.1107.5.2.32.35049.2015050810251346812...,0.12300,Pass
2,8192087,619278,V03,DCM_2015-05-08_ImagingUpload-14-14-grO2zP.tar,1.3.12.2.1107.5.2.32.35049.2015050810333048296...,0.02000,Pass
3,8192087,619278,V03,DCM_2015-05-08_ImagingUpload-14-14-grO2zP.tar,1.3.12.2.1107.5.2.32.35049.2015050810372458752...,0.09600,Pass
4,8192087,619278,V03,DCM_2015-05-08_ImagingUpload-14-14-grO2zP.tar,1.3.12.2.1107.5.2.32.35049.2015050810425167447...,0.09600,Pass
...,...,...,...,...,...,...,...
3370,3494204,337021,V10,DCM_2018-04-04_ImagingUpload-11-16-ioLmAM.tar,1.3.12.2.1107.5.2.43.167006.201804041514566309...,0.09300,Pass
3371,3494204,337021,V10,DCM_2018-04-04_ImagingUpload-11-16-ioLmAM.tar,1.3.12.2.1107.5.2.43.167006.201804041518353021...,0.12000,Pass
3372,3494204,337021,V10,DCM_2018-04-04_ImagingUpload-11-16-ioLmAM.tar,1.3.12.2.1107.5.2.43.167006.201804041520034816...,0.02000,Pass
3373,3494204,337021,V10,DCM_2018-04-04_ImagingUpload-11-16-ioLmAM.tar,1.3.12.2.1107.5.2.43.167006.201804041520034816...,0.02000,NaN


In [96]:
series_uids_pscid = df(tuple(((str(grp), str(series_uids.groupby('PSCID').get_group(
                                grp)['CandID'].unique().tolist()[0])),
                              series_uids.groupby('PSCID').get_group(grp).iloc[:, 2:])
                             for grp in series_uids.groupby('PSCID').groups))
series_uids_pscid

,0,1
0,"(3002498, 327986)",Visit ...
1,"(3025432, 658178)",Visit ...
2,"(3123186, 920577)",Visit ...
3,"(3149469, 790489)",Visit ...
4,"(3163875, 199801)",Visit ...
...,...,...
173,"(9322528, 401205)",Visit ...
174,"(9417188, 560180)",Visit ...
175,"(9494076, 630120)",Visit ...
176,"(9648306, 467954)",Visit ...


In [103]:
series_uids_pscid_dict = repr(series_uids_pscid.set_index(0).T.to_dict())
series_uids_pscid_dict                       

"{('3002498', '327986'): {1:      Visit                                        Archive  \\\n2245   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2246   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2247   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2248   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2249   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2250   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2251   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2252   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2253   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2254   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2255   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2256   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2257   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2258   V10  DCM_2017-05-15_ImagingUpload-12-57-ik8wkr.tar   \n2259   V10  DCM_2017-05-15_ImagingUpload

In [105]:
Path('~/series_uids_pscid2.json').write_text(
     series_uids_pscid_dict,
     encoding = 'UTF-8')


482818

In [106]:
series_uids_pscid.to_csv('~/series_uids_pscid.tsv',
                         sep = '\t',
                         encoding = 'UTF-8')

In [107]:
series_uids.to_csv('~/series_uids.tsv',
                   sep = '\t',
                   encoding = 'UTF-8')

sourcedata_path=Path(pjoin(dname(dname(datalad_config_path)),
                'sourcedata/cimaq_dicoms_20190901_v3.tar.gz'))

sourcedata_path.parent.as_posix()

mytar=tarfile.open(sourcedata_path)
tarfile_contents = get_fulltarcontents(mytar)

tarfile_contents['names'] = [str(aname) for aname in
                             tarfile_contents.name]
tarfile_contents['basenames'] = [bname(name).strip() for name
                                 in tarfile_contents.names]
tarfile_contents['split_basenames'] = [dict(tuple(zip(('dates', 'archv_names'),
                                                 name.split('_')[1:])))
                                       for name in tarfile_contents.basenames]
tarfile_contents['split_basenames']


1      {'dates': '2015-05-08', 'archv_names': 'Imagin...
2        {'dates': '2015-05-12', 'archv_names': 'CIMAQ'}
3        {'dates': '2015-05-12', 'archv_names': 'CIMAQ'}
4      {'dates': '2015-05-21', 'archv_names': 'Imagin...
5      {'dates': '2015-05-22', 'archv_names': 'Imagin...
                             ...                        
215    {'dates': '2019-05-30', 'archv_names': 'Imagin...
216    {'dates': '2019-05-30', 'archv_names': 'Imagin...
217    {'dates': '2019-06-14', 'archv_names': 'Imagin...
218    {'dates': '2019-07-04', 'archv_names': 'Imagin...
219    {'dates': '2019-07-11', 'archv_names': 'Imagin...
Name: split_basenames, Length: 219, dtype: object

In [84]:
tarfile_contents.to_csv('~/tarfile_contents.tsv',
                        sep = '\t',
                        encoding = 'UTF-8')

In [148]:
tarfile_contents['inner_contents'] = \
    [df.from_dict(mytar.getmember(row[1].names).get_info(),
                  orient = 'index').T.squeeze()
     for row in tarfile_contents.iterrows()]

In [5]:
load_utils_path = Path(xpu('~/py369env/bin/load_utils'))
load_utils_path
sorted(ls(load_utils_path))

['.git',
 'LICENSE',
 'README.md',
 '__init__.py',
 '__pycache__',
 'evenodd.py',
 'filterlist_exc.py',
 'filterlist_inc.py',
 'find_key.py',
 'flatten.py',
 'get_dst_path.py',
 'load_module.py',
 'loadfiles.py',
 'loadimages.py',
 'megamerge.py',
 'setup.py']

In [ ]:
#!usr/bin/python3,
 
import os
from typing import Union

def loadimages(indir: Union[os.PathLike, str]) -> list:
    """  Lists the full relative path of all \
         files in directory.,
         Only lists files, not directories.
         ----------------------------------
         Parameters,
         ----------,
         imdir: type = str,
             Name of the directory containing the images.

         Return,
         ------,
         imlist: type = list,
         1D list containing all \.jpeg\ files\ full relative paths 
     """
    imlist = []
    for allimages in os.walk(indir):
        for image in allimages[2]:
            indir = os.path.join(allimages[0], image)
            if os.path.isfile(indir):
                imlist.append(indir)
    return imlist

def main():
    if __name__ == __main__:
        loadimages(indir)


In [26]:
lu_path = xpu('~/py369env/bin/load_utils')
snif_path = xpu('~/py369env/bin/load_utils/sniffbytes')
szpath = snif_path = xpu('~/py369env/bin/load_utils/scanzip')



In [110]:
%pip install ~/py369env/bin/sniffbytes

Defaulting to user installation because normal site-packages is not writeable
Processing ./py369env/bin/sniffbytes
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for sniffbytes-francois.nadeau.1: filename=sniffbytes_francois.nadeau.1-0.0.1-py3-none-any.whl size=2460 sha256=ada7e58d85d2af7900a868eef8435525270b8235fd0b41973d987384cd77d2e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-us4gkoog/wheels/cf/ac/9d/3b311786189d52eeeab8f481fe19dff002f06b6b2b4b318bee
Successfully built sniffbytes-francois.nadeau.1
  Attempting uninstall: sniffbytes-francois.nadeau.1
    Found existing installation: sniffbytes-francois.nadeau.1 0.0.1
    Uninstalling sniffbytes-francois.nadeau.1-0.0.1:
      Successfully uninstalled sniffbytes-francois.nadeau.1-0.0.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated p

In [111]:
import sniffbytes as snif

ModuleNotFoundError: No module named 'sniffbytes'